## 1.Load train_Data from CSV

In [42]:
import pandas as pd

def load_data(path):
    """Loads data from a CSV file."""
    try:
        df = pd.read_csv(path)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == '__main__':
    data_path = 'train.csv'
    df = load_data(data_path)
    if df is not None:
        print(df.head())


   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

## 2. Fetch US Holiday Data from API

In [44]:
import requests
import pandas as pd

api_key = "URbJpoUsc2yXoYcezjO3ZttvcNEMWqrs"

def get_us_holidays(year):
    url = f"https://calendarific.com/api/v2/holidays?&api_key={api_key}&country=US&year={year}&type=national"
    response = requests.get(url)
    data = response.json()
    
    holidays = [
        {
            "date": item["date"]["iso"],
            "holiday_name": item["name"],
            "holiday_type": ", ".join(item["type"])
        }
        for item in data["response"]["holidays"]
    ]
    return pd.DataFrame(holidays)

# Fetch holidays from 2015 to 2018 (you can change the range)
all_years = []
for year in range(2015, 2019):
    df = get_us_holidays(year)
    all_years.append(df)

# Combine all holiday data
holidays_df = pd.concat(all_years, ignore_index=True)
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
holidays_df.head()


,date,holiday_name,holiday_type
0,2015-01-01,New Year's Day,National holiday
1,2015-01-19,Martin Luther King Jr. Day,National holiday
2,2015-02-16,Presidents' Day,National holiday
3,2015-05-25,Memorial Day,National holiday
4,2015-07-03,Independence Day (substitute),National holiday


In [45]:
# Save to CSV
holidays_df.to_csv('us_holidays_2015_2018.csv', index=False)


## 3. Extract CPI Data from Website

In [47]:
import pandas as pd

# URL with historical CPI table (InflationData.com)
url = "https://www.inflationdata.com/Inflation/Consumer_Price_Index/HistoricalCPI.aspx"

# Read all tables
tables = pd.read_html(url)

# Show how many tables were found
print(f"Found {len(tables)} tables")

# Preview all tables to find the right one
for i, table in enumerate(tables):
    print(f"\nTable {i}")
    print(table.head())


Found 1 tables

Table 0
   Year      Jan      Feb      Mar      Apr      May      Jun      Jul  \
0  2025  317.671  319.082  319.799      NaN      NaN      NaN      NaN   
1  2024  308.417  310.326  312.332  313.548  314.069  314.175  314.540   
2  2023  299.170  300.840  301.836  303.363  304.127  305.109  305.691   
3  2022  281.148  283.716  287.504  289.109  292.296  296.311  296.276   
4  2021  261.582  263.014  264.877  267.054  269.195  271.696  273.003   

       Aug      Sep      Oct      Nov      Dec     Ave.  
0      NaN      NaN      NaN      NaN      NaN      NaN  
1  314.796  315.301  315.664  315.493  315.605  313.689  
2  307.026  307.789  307.671  307.051  306.746  304.702  
3  296.171  296.808  298.012  297.711  296.797  292.655  
4  273.567  274.310  276.589  277.948  278.802  270.970  


##  Clean and Save CPI Data

In [49]:
import pandas as pd

# Step 1: Read CPI table
url = "https://www.inflationdata.com/Inflation/Consumer_Price_Index/HistoricalCPI.aspx"
tables = pd.read_html(url)
cpi_table = tables[0]

# Step 2: Remove the "Ave." column if it exists
if "Ave." in cpi_table.columns:
    cpi_table = cpi_table.drop(columns=["Ave."])

# Step 3: Melt the table (Year-Month-CPI)
cpi_long = cpi_table.melt(id_vars=["Year"], var_name="Month", value_name="CPI")

# Step 4: Drop missing CPI rows
cpi_cleaned = cpi_long.dropna(subset=["CPI"]).copy()

# Step 5: Clean up and convert types
cpi_cleaned["Year"] = cpi_cleaned["Year"].astype(int)
cpi_cleaned["Month"] = cpi_cleaned["Month"].str.strip()
cpi_cleaned["CPI"] = pd.to_numeric(cpi_cleaned["CPI"], errors="coerce")

# Step 6: Create a datetime column
cpi_cleaned["Date"] = pd.to_datetime(cpi_cleaned["Month"] + " " + cpi_cleaned["Year"].astype(str), format="%b %Y")

# Step 7: Save cleaned data
cpi_cleaned[["Date", "CPI"]].to_csv("cleaned_cpi_data.csv", index=False)

print("✅ Cleaned CPI data saved as 'cleaned_cpi_data.csv'")


✅ Cleaned CPI data saved as 'cleaned_cpi_data.csv'


##  Load and Inspect Training Data


In [51]:
import pandas as pd

# Load train.csv
df_main = pd.read_csv("train.csv")

# Show basic info
print(df_main.info())
df_main.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## Preprocess Training Data

In [53]:
# 1. Convert date columns to datetime
df_main["Order Date"] = pd.to_datetime(df_main["Order Date"], dayfirst=True)
df_main["Ship Date"] = pd.to_datetime(df_main["Ship Date"], dayfirst=True)

# 2. Handle missing Postal Codes
df_main["Postal Code"] = df_main["Postal Code"].fillna(-1).astype(int)
print("✅ Null postal codes handled.")

# 3. Drop irrelevant columns
df_main_cleaned = df_main.drop(["Row ID", "Order ID", "Customer ID", "Product ID"], axis=1)

# 4. Extract year and month
df_main_cleaned["Order_Year"] = df_main_cleaned["Order Date"].dt.year
df_main_cleaned["Order_Month"] = df_main_cleaned["Order Date"].dt.month

# 5. Check and remove duplicates
dup_count_before = df_main_cleaned.duplicated().sum()
print(f"🧹 Duplicate rows before: {dup_count_before}")
df_main_cleaned = df_main_cleaned.drop_duplicates()
dup_count_after = df_main_cleaned.duplicated().sum()
print(f"✅ Duplicate rows after: {dup_count_after}")

# 6. Preview cleaned dataset
df_main_cleaned.head()


✅ Null postal codes handled.
🧹 Duplicate rows before: 1
✅ Duplicate rows after: 0


,Order Date,Ship Date,Ship Mode,Customer Name,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Product Name,Sales,Order_Year,Order_Month
0,2017-11-08,2017-11-11,Second Class,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2017,11
1,2017-11-08,2017-11-11,Second Class,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,2017,11
2,2017-06-12,2017-06-16,Second Class,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2017,6
3,2016-10-11,2016-10-18,Standard Class,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,2016,10
4,2016-10-11,2016-10-18,Standard Class,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2016,10


## 4. Load and Preview Holiday Data

In [55]:
# Load holidays data
df_holidays = pd.read_csv("us_holidays_2015_2018.csv")

# Preview the dataset
print("🗓️ Shape:", df_holidays.shape)
print("🧠 Columns:", df_holidays.columns.tolist())
df_holidays.head()


🗓️ Shape: (47, 3)
🧠 Columns: ['date', 'holiday_name', 'holiday_type']


,date,holiday_name,holiday_type
0,2015-01-01,New Year's Day,National holiday
1,2015-01-19,Martin Luther King Jr. Day,National holiday
2,2015-02-16,Presidents' Day,National holiday
3,2015-05-25,Memorial Day,National holiday
4,2015-07-03,Independence Day (substitute),National holiday


##  Preprocess Holiday Data

In [57]:

# 1. Convert 'date' column to datetime format
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# 2. Standardize column names (optional but useful)
df_holidays.columns = df_holidays.columns.str.strip().str.lower().str.replace(" ", "_")

# 3. Remove any duplicate rows (if any)
df_holidays = df_holidays.drop_duplicates()

# 4. Check for null values
print("🧼 Null values:\n", df_holidays.isnull().sum())

# 5. Final preview
df_holidays.head()


🧼 Null values:
 date            0
holiday_name    0
holiday_type    0
dtype: int64


,date,holiday_name,holiday_type
0,2015-01-01,New Year's Day,National holiday
1,2015-01-19,Martin Luther King Jr. Day,National holiday
2,2015-02-16,Presidents' Day,National holiday
3,2015-05-25,Memorial Day,National holiday
4,2015-07-03,Independence Day (substitute),National holiday


## 5. Load and Inspect COVID-19 Data


In [59]:
import pandas as pd

df_covid = pd.read_csv("daily-new-confirmed-covid-19-deaths-per-million-people (2).csv")
df_covid.info()
df_covid.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473197 entries, 0 to 473196
Data columns (total 3 columns):
 #   Column                                                                                                Non-Null Count   Dtype  
---  ------                                                                                                --------------   -----  
 0   Entity                                                                                                473197 non-null  object 
 1   Day                                                                                                   473197 non-null  object 
 2   Daily new confirmed deaths due to COVID-19 per million people (rolling 7-day average, right-aligned)  473197 non-null  float64
dtypes: float64(1), object(2)
memory usage: 10.8+ MB


,Entity,Day,"Daily new confirmed deaths due to COVID-19 per million people (rolling 7-day average, right-aligned)"
0,Afghanistan,2020-01-09,0.0
1,Afghanistan,2020-01-10,0.0
2,Afghanistan,2020-01-11,0.0
3,Afghanistan,2020-01-12,0.0
4,Afghanistan,2020-01-13,0.0


In [60]:
# Rename columns for simplicity
df_covid.columns = ['Country', 'Date', 'Deaths_per_million']

# Convert date to datetime format
df_covid['Date'] = pd.to_datetime(df_covid['Date'])

# Filter data for the United States
df_covid_usa = df_covid[df_covid['Country'] == 'United States']

# Reset index and drop the original country column (if not needed later)
df_covid_usa = df_covid_usa[['Date', 'Deaths_per_million']].reset_index(drop=True)

df_covid_usa.head()


,Date,Deaths_per_million
0,2020-01-09,0.0
1,2020-01-10,0.0
2,2020-01-11,0.0
3,2020-01-12,0.0
4,2020-01-13,0.0


## 6. Load and Inspect Trends Data

In [62]:
df_trends = pd.read_csv("trends.csv")
df_trends.info()
df_trends.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26955 entries, 0 to 26954
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  26955 non-null  object
 1   year      26955 non-null  int64 
 2   category  26955 non-null  object
 3   rank      26955 non-null  int64 
 4   query     26955 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.0+ MB


,location,year,category,rank,query
0,Global,2001,Consumer Brands,1,Nokia
1,Global,2001,Consumer Brands,2,Sony
2,Global,2001,Consumer Brands,3,BMW
3,Global,2001,Consumer Brands,4,Palm
4,Global,2001,Consumer Brands,5,Adobe


##  Preprocess Trends Data

In [64]:
# Step 1: Clean column names
df_trends.columns = df_trends.columns.str.lower().str.replace(' ', '_')

# Step 2: Drop duplicates
df_trends.drop_duplicates(inplace=True)

# Step 3: Filter for United States
df_trends_us = df_trends[df_trends['location'] == 'United States'].copy()

# Optional: Check unique categories or year ranges
print(df_trends_us['category'].unique())
print(df_trends_us['year'].value_counts().sort_index())


['Movie Showtimes' 'Scandals' 'Tickets' 'Deaths of 2007' 'Diets' 'Fitness'
 'How To' 'Lawsuits' 'Lyrics' 'Presidential Campaign' 'Recipes'
 'Ringtones' 'What is' 'Who is' "It's the Economy" 'Books'
 'Google.com Fastest Falling' 'Google.com Fastest Rising' 'Images'
 'Maps Searches' 'Mobile Searches' 'News Searches' 'Baseball' 'Basketball'
 'Cars' 'Costumes' 'Dance Moves' 'DIY' 'Donate To' 'Economy'
 'Falling Searches' 'Football' 'Gadgets' 'Google Image Searches'
 'Google Maps Searches' 'Google News Searches' 'Home Appliances'
 'Literature' 'News Sources' 'Political Scandals' 'Politics'
 'Pop Culture / People' 'Pop Culture / Songs' 'Pop Culture / Toys'
 'Pop Culture / TV Shows' 'Product Searches' 'Quirky Environmental '
 'Recalls' 'Science' 'Soccer' 'Sports' 'Technology' 'Top Trending'
 'Workouts' 'Annual Events of 2012' 'Apparel Brands' 'Athletes' 'Authors'
 'Calorie Searches' 'Colleges & Universities' 'Donations'
 'Election Issues' 'Endangered Species' 'Events of 2012' 'Food Delivery'


In [65]:
df_trends_us['category'] = df_trends_us['category'].str.strip().str.lower()

# Example replacements (you can expand this dict as needed)
replace_map = {
    'how to...': 'how to',
    'how to': 'how to',
    'what is...?': 'what is',
    'what is...': 'what is',
    'gif': 'gifs',
    'gifs': 'gifs',
    'songs/lyrics': 'songs',
    '¿qué es...?': 'what is',
    'qué es...?': 'what is',
    'fashion style searches': 'fashion',
    'fashion questions': 'fashion',
    'fashion brands': 'fashion',
}

df_trends_us['category'] = df_trends_us['category'].replace(replace_map)


## 7. Load and Inspect Reddit Data

In [67]:
df_reddit=pd.read_csv("Reddit_Data.csv")
df_reddit.info()
df_reddit.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


## Preprocess Reddit Data

In [69]:
# Drop rows where comment is missing
df_reddit_cleaned = df_reddit.dropna(subset=["clean_comment"])

# Optional: Clean extra whitespace, lowercase, etc.
df_reddit_cleaned.loc[:, "clean_comment"] = df_reddit_cleaned["clean_comment"].str.strip().str.lower()


# Optional: Check distribution of categories
category_counts = df_reddit_cleaned["category"].value_counts()
print(category_counts)


category
 1    15830
 0    13042
-1     8277
Name: count, dtype: int64


## 8. Load and Inspect Twitter Data

In [71]:
df_tweets=pd.read_csv("Twitter_Data.csv")
df_tweets.info()
df_tweets.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


##  Preprocess Twitter Data

In [73]:
# Drop missing
df_tweets_cleaned = df_tweets.dropna(subset=["clean_text", "category"]).copy()

# Convert category to int
df_tweets_cleaned["category"] = df_tweets_cleaned["category"].astype(int)

# Clean text: strip + lowercase
df_tweets_cleaned["clean_text"] = df_tweets_cleaned["clean_text"].str.strip().str.lower()


##  Combine and Process Sentiment Data


In [75]:
import pandas as pd

# ✅ Step 1: Rename columns safely by creating new DataFrames
df_reddit_cleaned = df_reddit_cleaned.rename(columns={"clean_comment": "text"})
df_tweets_cleaned = df_tweets_cleaned.rename(columns={"clean_text": "text"})

# ✅ Step 2: Convert 'category' to numeric (force errors to NaN), then drop any NaNs
df_reddit_cleaned["category"] = pd.to_numeric(df_reddit_cleaned["category"], errors="coerce")
df_tweets_cleaned["category"] = pd.to_numeric(df_tweets_cleaned["category"], errors="coerce")

df_reddit_cleaned = df_reddit_cleaned.dropna(subset=["text", "category"])
df_tweets_cleaned = df_tweets_cleaned.dropna(subset=["text", "category"])

# ✅ Step 3: Convert 'category' to integer type explicitly
df_reddit_cleaned["category"] = df_reddit_cleaned["category"].astype(int)
df_tweets_cleaned["category"] = df_tweets_cleaned["category"].astype(int)

# ✅ Step 4: Concatenate both cleaned datasets into one
df_sentiment_combined = pd.concat(
    [df_reddit_cleaned[["text", "category"]],
     df_tweets_cleaned[["text", "category"]]],
    ignore_index=True
)

# ✅ Step 5: Overview
print(df_sentiment_combined.info())
print(df_sentiment_combined["category"].value_counts())
print(df_sentiment_combined.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200118 entries, 0 to 200117
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      200118 non-null  object
 1   category  200118 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 2.3+ MB
None
category
 1    88079
 0    68253
-1    43786
Name: count, dtype: int64
                                                text  category
0  family mormon have never tried explain them th...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1


##  Merge Holiday Data with Main Dataset

In [77]:

# 2. Merge with main dataset on Order Date and Holiday date
df_merged = df_main_cleaned.merge(
    df_holidays,
    how="left",
    left_on="Order Date",
    right_on="date"
)

# 3. Drop the 'date' column after merge (already have Order Date)
df_merged.drop(columns=["date"], inplace=True)

# 4. Fill missing holiday names/types with "No Holiday"
df_merged["holiday_name"] = df_merged["holiday_name"].fillna("No Holiday")
df_merged["holiday_type"] = df_merged["holiday_type"].fillna("No Holiday")

# 5. Final check
print("✅ Merged shape:", df_merged.shape)
print(df_merged[["Order Date", "holiday_name", "holiday_type"]].sample(5))


✅ Merged shape: (9799, 18)
     Order Date holiday_name holiday_type
444  2017-09-05   No Holiday   No Holiday
8547 2016-12-05   No Holiday   No Holiday
3321 2015-12-20   No Holiday   No Holiday
8337 2015-12-27   No Holiday   No Holiday
3095 2016-08-23   No Holiday   No Holiday


##  Extract Cities and Date Range from Main Dataset


In [79]:
# Extract unique cities from your main dataset
unique_cities = df_main_cleaned['City'].dropna().unique()
print(f"📍 Total cities: {len(unique_cities)}")
print(unique_cities[:10])  # Preview first few cities

# Get date range
min_date = df_main_cleaned['Order Date'].min().strftime('%Y-%m-%d')
max_date = df_main_cleaned['Order Date'].max().strftime('%Y-%m-%d')
print(f"📆 Date range: {min_date} to {max_date}")


📍 Total cities: 529
['Henderson' 'Los Angeles' 'Fort Lauderdale' 'Concord' 'Seattle'
 'Fort Worth' 'Madison' 'West Jordan' 'San Francisco' 'Fremont']
📆 Date range: 2015-01-03 to 2018-12-30


##  Identify Top 20 Cities by Order Frequency

In [81]:
# Top 20 cities by frequency in order data
top_cities = df_main_cleaned['City'].value_counts().head(20).index.tolist()
print(top_cities)


['New York City', 'Los Angeles', 'Philadelphia', 'San Francisco', 'Seattle', 'Houston', 'Chicago', 'Columbus', 'San Diego', 'Springfield', 'Dallas', 'Jacksonville', 'Detroit', 'Newark', 'Jackson', 'Columbia', 'Richmond', 'Aurora', 'Phoenix', 'Arlington']


##  List DataFrame Names and Columns

In [83]:
for name in dir():
    if isinstance(eval(name), pd.DataFrame):
        print(f"{name}: {eval(name).columns.tolist()}")


_13: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']
_15: ['Order Date', 'Ship Date', 'Ship Mode', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Order_Year', 'Order_Month']
_17: ['date', 'holiday_name', 'holiday_type']
_19: ['date', 'holiday_name', 'holiday_type']
_44: ['date', 'holiday_name', 'holiday_type']
_51: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']
_53: ['Order Date', 'Ship Date', 'Ship Mode', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Order_Year

##  Convert Order Date to datetime and Add Year Column


In [85]:
# Convert 'Order Date' to datetime
df_merged["Order Date"] = pd.to_datetime(df_merged["Order Date"])

# Add year column
df_merged["year"] = df_merged["Order Date"].dt.year


##  Convert Year Column to Integer


In [87]:
df_trends_us["year"] = df_trends_us["year"].astype(int)


##  Merge Main and Trends Data


In [89]:
df_final = pd.merge(df_merged, df_trends_us, on="year", how="left")


##  Inspect Merged DataFrame


In [91]:
df_final.head()
df_final.columns
df_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1794015 entries, 0 to 1794014
Data columns (total 23 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Order Date     datetime64[ns]
 1   Ship Date      datetime64[ns]
 2   Ship Mode      object        
 3   Customer Name  object        
 4   Segment        object        
 5   Country        object        
 6   City           object        
 7   State          object        
 8   Postal Code    int32         
 9   Region         object        
 10  Category       object        
 11  Sub-Category   object        
 12  Product Name   object        
 13  Sales          float64       
 14  Order_Year     int32         
 15  Order_Month    int32         
 16  holiday_name   object        
 17  holiday_type   object        
 18  year           int32         
 19  location       object        
 20  category       object        
 21  rank           int64         
 22  query          object        
dtypes: date

In [92]:
df_final.isnull().sum()


Order Date       0
Ship Date        0
Ship Mode        0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Order_Year       0
Order_Month      0
holiday_name     0
holiday_type     0
year             0
location         0
category         0
rank             0
query            0
dtype: int64

##  Merge COVID-19 Data with Main DataFrame


In [94]:
# Step 1: Create a new column 'Order_Date_only' to match COVID Date granularity
df_final['Order_Date_only'] = df_final['Order Date'].dt.date

# Step 2: Also extract date only (not datetime) in df_covid_usa
df_covid_usa['Date_only'] = df_covid_usa['Date'].dt.date

# Step 3: Merge on date
df_final = df_final.merge(
    df_covid_usa[['Date_only', 'Deaths_per_million']],
    left_on='Order_Date_only',
    right_on='Date_only',
    how='left'
)

# Step 4: Drop helper columns if you want
df_final.drop(columns=['Order_Date_only', 'Date_only'], inplace=True)


In [95]:
print(df_final[['Order Date', 'Deaths_per_million']].head())


  Order Date  Deaths_per_million
0 2017-11-08                 NaN
1 2017-11-08                 NaN
2 2017-11-08                 NaN
3 2017-11-08                 NaN
4 2017-11-08                 NaN


##  Merge CPI Data with Main DataFrame


In [97]:
# Step 1: Create a clean date-only column for merging
df_final['Order_Date_only'] = df_final['Order Date'].dt.to_period('M').dt.to_timestamp()

# Step 2: Round CPI Date to first of month (if needed, already should be in 'YYYY-MM-01')
cpi_cleaned['CPI_Date'] = cpi_cleaned['Date'].dt.to_period('M').dt.to_timestamp()

# Step 3: Merge on these normalized month-dates
df_final = df_final.merge(
    cpi_cleaned[['CPI_Date', 'CPI']],
    left_on='Order_Date_only',
    right_on='CPI_Date',
    how='left'
)

# Step 4: Drop helper columns
df_final.drop(columns=['Order_Date_only', 'CPI_Date'], inplace=True)


In [98]:
print(df_final[['Order Date', 'CPI']].head())


  Order Date      CPI
0 2017-11-08  246.669
1 2017-11-08  246.669
2 2017-11-08  246.669
3 2017-11-08  246.669
4 2017-11-08  246.669


##  9. Load and Clean Gas Price Data


In [100]:
import pandas as pd
from pandas import PeriodIndex

# Load dataset, skipping first 5 rows and setting column names
gas_prices_df = pd.read_csv(
    "Weekly_U.S._Regular_All_Formulations_Retail_Gasoline_Prices.csv",
    skiprows=5,
    names=["Date", "Price"]
)

# Convert Date column to datetime
gas_prices_df["Date"] = pd.to_datetime(gas_prices_df["Date"], errors='coerce')

# Drop rows with missing Date or Price
gas_prices_df.dropna(subset=["Date", "Price"], inplace=True)

# Convert Price to numeric
gas_prices_df["Price"] = pd.to_numeric(gas_prices_df["Price"], errors="coerce")

# Drop rows with missing or invalid Price
gas_prices_df.dropna(subset=["Price"], inplace=True)

print("✅ Gas data cleaned. Shape:", gas_prices_df.shape)
print(gas_prices_df.head())


✅ Gas data cleaned. Shape: (1803, 2)
        Date  Price
0 2025-04-14  3.168
1 2025-04-07  3.243
2 2025-03-31  3.162
3 2025-03-24  3.115
4 2025-03-17  3.058


##  Add Weekly Period Column to DataFrames


In [102]:
# Ensure 'Order Date' in df_final is datetime
df_final["Order Date"] = pd.to_datetime(df_final["Order Date"], errors="coerce")

# Create weekly period columns
df_final["Order_Week"] = PeriodIndex(df_final["Order Date"], freq="W").start_time
gas_prices_df["Order_Week"] = PeriodIndex(gas_prices_df["Date"], freq="W").start_time

print("📅 Weekly column added to both DataFrames.")


📅 Weekly column added to both DataFrames.


##  Merge Gas Price Data into Main DataFrame


In [104]:
# Merge gas price data into df_final based on 'Order_Week'
df_final = pd.merge(df_final, gas_prices_df[["Order_Week", "Price"]], on="Order_Week", how="left")

# Optional: Rename for clarity
df_final.rename(columns={"Price": "Gasoline_Price"}, inplace=True)

print("✅ Merged final shape:", df_final.shape)
print(df_final[["Order Date", "Order_Week", "Gasoline_Price"]].head())


✅ Merged final shape: (1794015, 27)
  Order Date Order_Week  Gasoline_Price
0 2017-11-08 2017-11-06           2.561
1 2017-11-08 2017-11-06           2.561
2 2017-11-08 2017-11-06           2.561
3 2017-11-08 2017-11-06           2.561
4 2017-11-08 2017-11-06           2.561


##  Save Final Dataset to CSV


In [106]:

df_final.to_csv("retail_enriched_dataset.csv", index=False)

print("✅ Final dataset saved as 'retail_enriched_dataset.csv'")


✅ Final dataset saved as 'retail_enriched_dataset.csv'
